In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, f1_score

import keras
import tensorflow as tf

In [3]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
print('length of data:',len(df_train))
df_train.isnull().sum()

length of data: 7613


id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [5]:
df_train = df_train.drop(['id','keyword','location'],axis=1)
df_train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
df_train.shape

(7613, 2)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_train['text'], df_train['target'], stratify=df_train['target'], test_size=0.3, random_state=42)


In [12]:
#Create Document Term Matrix
vectorizer = CountVectorizer(stop_words='english', lowercase=True, token_pattern=r'(?u)\b[A-Za-z]+\b')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [13]:
#Prediction using random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42).fit(X_train, y_train)

# Predict sentiment on the test set
predictions_rf = rf.predict(X_test)

# Calculate evaluation metrics
print("Accuracy of Random Forest Classifier:", accuracy_score(y_test, predictions_rf))

Accuracy of Random Forest Classifier: 0.7845884413309983


In [98]:
# import tpot
from tpot import TPOTClassifier

tpot = TPOTClassifier(config_dict='TPOT sparse', generations=5, population_size=20, cv=5, random_state=42, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7834456394401431

Generation 2 - Current best internal CV score: 0.7834456394401431

Generation 3 - Current best internal CV score: 0.7834456394401431

Generation 4 - Current best internal CV score: 0.79095438169983

Generation 5 - Current best internal CV score: 0.7915154718177734

Best pipeline: LogisticRegression(input_matrix, C=1.0, dual=False, penalty=l2)
0.797723292469352


In [28]:
X_train_final = vectorizer.fit_transform(df_train['text'])
X_test_final = vectorizer.transform(df_test['text'])
y_train_final = df_train['target']

In [29]:
rf_final = RandomForestClassifier(random_state=42).fit(X_train_final, y_train_final)

# Predict sentiment on the test set
ypred = rf_final.predict(X_test_final)
final = np.round(ypred)
output = pd.DataFrame({'id': df_test.id, 'target': final})
output.to_csv('submission.csv', index=False)